In [1]:
spark
import numpy as np
import pandas as pd
import re
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import trim,ltrim,rtrim
from pyspark.sql.functions import *
from pyspark.sql.functions import mean
from ydata_profiling import ProfileReport
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import lit,col,isnan
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.functions import mean, when, col
from pyspark.ml.classification import DecisionTreeClassificationModel
from pycaret.classification import *
from pycaret.classification import ClassificationExperiment
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

Intitializing PySpark ...

23/06/01 20:37:57 WARN Utils: Your hostname, DESKTOP-VJTDOFP resolves to a loopback address: 127.0.1.1; using 172.27.220.32 instead (on interface eth0)
23/06/01 20:37:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/06/01 20:37:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Web UI available at http://172.27.220.32:4040
SparkContext available as 'sc' (version = 3.3.1, master = local[*], app id = local-1685648279652)
SparkSession available as 'spark'


In [2]:
path_games = "./trabalho_def_table_games.csv"
path_paises = "./trabalho_def_table_paises.csv"
path_players = "./trabalho_def_table_players.csv"
path_tornaments = "./trabalho_def_table_tournaments.csv"


games_df = spark.read.option('header' , 'True').csv(path_games)

paises_df = spark.read.option('header' , 'True').csv(path_paises)

players_df = spark.read.option('header' , 'True').csv(path_players)

tournaments_df = spark.read.option('header' , 'True').csv(path_tornaments)

df_1 = spark.read.json("./atpplayers.json")
df_2 = spark.read.json("./atpplayers.json")
df_mt = spark.read.json("./atpplayers.json")

In [3]:
colunas_players = df_2.columns

for coluna in colunas_players:
    
    winner = coluna + "_2"
    
    df_2 = df_2.withColumnRenamed(coluna,winner)
    
                                       
    

In [4]:
df_3 = df_1.join(df_2, (df_1.PlayerName == df_2.Oponent_2 )& (df_1.Date == df_2.Date_2 ) & (df_1.Tournament == df_2.Tournament_2)& (df_1.GameRound == df_2.GameRound_2),"inner" )

In [5]:
df_4 = df_3.select(df_3.Date,df_3.GameRound,df_3.Tournament,df_3.Oponent,df_3.GameRank,df_3.Oponent_2,df_3.GameRank_2)

In [6]:
games = games_df.select("Score").collect()
jogos = []
sets = []
for i in range (0,len(games)):
    i = games[i]["Score"]
   
    
    jogo = re.findall(r'\d\d', str(i) )
    
    if isinstance(jogo, list):
        jogos.append(jogo)
        sets.append(len(jogo))
    else:
        jogos.append([jogo])
        n_sets = [jogo]
        sets.append(len(n_sets))
   

    


In [7]:
games_df_pandas = games_df.toPandas()



In [8]:
games_df_pandas["nSets"] = sets
games_df_pandas["Sets"] = jogos


new_games_df = spark.createDataFrame(games_df_pandas)

In [9]:
players_df = players_df.where(players_df.Name.isNotNull())



In [10]:
players_count = players_df.count()

In [11]:
paises_df = paises_df.where(paises_df.Codigo.isNotNull())

In [12]:
tournaments_df = tournaments_df.where(tournaments_df.Name.isNotNull())


In [13]:
lez_see = new_games_df.join(tournaments_df,new_games_df.tournament_id == tournaments_df.id,"inner")
new_games_df = lez_see.where(lez_see.country == "US")


In [14]:
new_games_df.count()

23/06/01 20:38:51 WARN TaskSetManager: Stage 13 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.


86147

In [15]:
new_games_df.show(1,vertical = True)

23/06/01 20:38:53 WARN TaskSetManager: Stage 17 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.
-RECORD 0-----------------------------
 GameRank      | 1187                 
 GameRound     | 1st Round Qualifying 
 Score         | 67,  67              
 tournament_id | 186                  
 winner_id     | 102                  
 loser_id      | 7274                 
 nSets         | 2                    
 Sets          | [67, 67]             
 id            | 186                  
 Date          | 2016.10.31 - 2016... 
 Ground        | Hard                 
 Location      | Charlottesville, ... 
 Name          | Charlottesville      
 Prize         | $50,000              
 country       | US                   
only showing top 1 row



In [16]:
new_games_df_ini = new_games_df

In [17]:
new_games_df_ini= new_games_df_ini.drop("nSets")

In [18]:
new_games_df_ini= new_games_df_ini.drop("Sets")

pandas_ini = new_games_df_ini.toPandas()

report = ProfileReport(pandas_ini)
report.to_file("report_ini.html")

In [19]:
df_4 = df_4.alias("df_4")
players_df = players_df.alias("players_df")


In [20]:
df_4 = df_4.join(players_df,df_4.Oponent == players_df.Name ).select("df_4.*",players_df.id)



In [21]:
df_4 = df_4.select(df_4.Date,df_4.Tournament,df_4.id,df_4.GameRank,df_4.Oponent)


In [22]:
df_4 = df_4.withColumn('GameRank', regexp_replace(df_4.GameRank, '[-]', "2300").cast('integer'))

In [23]:
df_4.groupby().max('GameRank').first().asDict()['max(GameRank)']

2300

In [24]:
df_4_winner = df_4
df_4_loser = df_4


colunas_players = df_4.columns

for coluna in colunas_players:
    
    winner = coluna + "_winner"
    loser = coluna + "_loser"
    
    df_4_loser = df_4_loser.withColumnRenamed(coluna,loser)
    df_4_winner = df_4_winner.withColumnRenamed(coluna,winner)
    
                                       
    




In [25]:
players_df_work = players_df


players_df_work = players_df_work.drop(players_df_work.Hand)
players_df_work = players_df_work.drop(players_df_work.Name)
players_df_work = players_df_work.drop(players_df_work.Link)
players_df_work = players_df_work.drop(players_df_work.Born)
    

colunas_players = players_df_work.columns


winner_df = players_df_work

loser_df = players_df_work



In [26]:
for coluna in colunas_players:
    
    winner = coluna + "_winner"
    loser = coluna + "_loser"
    
    winner_df = winner_df.withColumnRenamed(coluna,winner)
    loser_df = loser_df.withColumnRenamed(coluna,loser)
                                       
    

In [27]:
df_4_winner = df_4_winner.distinct()
df_4_loser = df_4_loser.distinct()

work_df = new_games_df

work_df.select(work_df.country).distinct().show()

23/06/01 20:39:00 WARN TaskSetManager: Stage 29 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.
+-------+
|country|
+-------+
|     US|
+-------+



In [28]:
work_df = work_df.alias("work_df")

In [29]:
work_df = work_df.join(df_4_winner, (work_df.winner_id == df_4_winner.id_winner)&(work_df.Date == df_4_winner.Date_winner)&(work_df.Name == df_4_winner.Tournament_winner),"left").select("work_df.*",df_4_winner.GameRank_winner)

In [30]:
work_df = work_df.alias("work_df")

In [31]:
work_df = work_df.join(df_4_loser, (work_df.loser_id == df_4_loser.id_loser)&(work_df.Date == df_4_loser.Date_loser)&(work_df.Name == df_4_loser.Tournament_loser),"left").select("work_df.*",df_4_loser.GameRank_loser)

In [32]:
work_df = work_df.join(winner_df, winner_df.id_winner == work_df.winner_id,"left")



In [33]:
work_df = work_df.join(loser_df, loser_df.id_loser == work_df.loser_id,"left")

In [34]:
work_df = work_df.withColumnRenamed("id","id_tournament")

In [35]:
df_count = work_df.count()

23/06/01 20:39:05 WARN TaskSetManager: Stage 37 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.


In [40]:
work_dfmt = work_df

In [41]:
work_dfmt.show(1,vertical = True)

23/06/01 20:39:49 WARN TaskSetManager: Stage 187 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.
-RECORD 0-------------------------------
 GameRank        | 39                   
 GameRound       | Round of 16          
 Score           | 16 36                
 tournament_id   | 17321                
 winner_id       | 3090                 
 loser_id        | 254                  
 nSets           | 2                    
 Sets            | [16, 36]             
 id_tournament   | 17321                
 Date            | 2006.08.21 - 2006... 
 Ground          | Hard                 
 Location        | New Haven, CT, U.... 
 Name            | New Haven            
 Prize           | $650,000             
 country         | US                   
 GameRank_winner | 39                   
 GameRank_loser  | 57                   
 id_winner       | 3090                 
 Height_winner   | 185                  
 mainhand_winner | Right-Handed      

In [42]:
work_df = work_df.drop(work_df.tournament_id)
work_df = work_df.drop(work_df.winner_id)
work_df = work_df.drop(work_df.loser_id)
work_df = work_df.drop(work_df.Score)


In [43]:
games = work_df.select("Date").collect()
datas = []

for i in range (0,len(games)):
    i = games[i]["Date"]
   
    
    data = re.findall(r'\d\d\d\d', str(i) )
    
    if isinstance(data, list):
        datas.append(data[0])
        
    else:
        datas.append(data)
 

23/06/01 20:39:57 WARN TaskSetManager: Stage 223 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.


In [44]:
work_df = work_df.drop(work_df.country)
work_df = work_df.drop(work_df.Location)

In [45]:
work_df_pandas = work_df.toPandas()

23/06/01 20:40:06 WARN TaskSetManager: Stage 250 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.


In [46]:
work_df_pandas["Year"] = datas

In [47]:
work_df = spark.createDataFrame(work_df_pandas)

In [48]:
work_df = work_df.drop(work_df.Date)

In [49]:
games = work_df.select("Name").collect()
is_slam = []
for i in range (0,len(games)):
    i = games[i]["Name"]
    
    if i == "US Open":
        is_slam.append(1)
    else:
        is_slam.append(0)
   


   

23/06/01 20:40:22 WARN TaskSetManager: Stage 274 contains a task of very large size (1653 KiB). The maximum recommended task size is 1000 KiB.


In [50]:
work_df = work_df.withColumn("Year",work_df.Year.cast('integer'))

In [51]:
work_df_pandas= work_df.toPandas()

23/06/01 20:40:23 WARN TaskSetManager: Stage 275 contains a task of very large size (1653 KiB). The maximum recommended task size is 1000 KiB.


In [52]:
work_df_pandas["is_slam"] = is_slam

In [53]:
work_df = spark.createDataFrame(work_df_pandas)



In [54]:
work_df = work_df.drop("Name")
work_df = work_df.drop("GameRank")


In [55]:

work_df = work_df.withColumn('Prize', regexp_replace(work_df.Prize, '[$,]', '').cast('double'))


In [56]:
work_df = work_df.withColumn("backhand_loser",ltrim(work_df.backhand_loser))

In [57]:
work_df = work_df.withColumn("backhand_winner",ltrim(work_df.backhand_winner))

In [58]:
work_df_pandas= work_df.toPandas()

23/06/01 20:40:33 WARN TaskSetManager: Stage 276 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


report = ProfileReport(work_df_pandas)
report.to_file("report.html")

In [59]:
work_df_v2 = work_df

In [60]:
work_df_v2.printSchema()

root
 |-- GameRound: string (nullable = true)
 |-- nSets: long (nullable = true)
 |-- Sets: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id_tournament: string (nullable = true)
 |-- Ground: string (nullable = true)
 |-- Prize: double (nullable = true)
 |-- GameRank_winner: double (nullable = true)
 |-- GameRank_loser: double (nullable = true)
 |-- id_winner: string (nullable = true)
 |-- Height_winner: string (nullable = true)
 |-- mainhand_winner: string (nullable = true)
 |-- backhand_winner: string (nullable = true)
 |-- country_winner: string (nullable = true)
 |-- id_loser: string (nullable = true)
 |-- Height_loser: string (nullable = true)
 |-- mainhand_loser: string (nullable = true)
 |-- backhand_loser: string (nullable = true)
 |-- country_loser: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- is_slam: long (nullable = true)



In [61]:
work_df_v2 = work_df_v2.withColumn("Height_winner",work_df_v2.Height_winner.cast('integer'))

In [62]:
work_df_v2 = work_df_v2.withColumn("Height_loser",work_df_v2.Height_loser.cast('integer'))

In [63]:
work_df_v2 = work_df_v2.withColumn("Height_dif",abs(work_df_v2.Height_winner -work_df_v2.Height_loser) )


In [64]:
work_df_v2 = work_df_v2.withColumn("winner_home", when(col("country_winner") == "US", 1).otherwise(0))
       


In [65]:
work_df_v2 = work_df_v2.withColumn("loser_home", when(col("country_loser") == "US", 1).otherwise(0))

In [66]:
work_df_v2.select(work_df_v2.mainhand_loser).distinct().show()
        
    
    

23/06/01 20:40:35 WARN TaskSetManager: Stage 277 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.
+--------------+
|mainhand_loser|
+--------------+
|  Ambidextrous|
|  Right-Handed|
|   Left-Handed|
|          NULL|
|          null|
|          null|
+--------------+



In [67]:
work_df_v2 = work_df_v2.withColumn("comb_hands", when((col("mainhand_loser") == "Right-Handed")&(col("mainhand_winner") == "Right-Handed"), "RR")\
                                   .when((col("mainhand_loser") == "Left-Handed")&(col("mainhand_winner") == "Left-Handed"), "LL")\
                                   .when((col("mainhand_loser") == "Right-Handed")&(col("mainhand_winner") == "Left-Handed"), "RL")\
                                   .when((col("mainhand_loser") == "Left-Handed")&(col("mainhand_winner") == "Right-Handed"), "RL")\
                                   .when((col("mainhand_loser") == "Ambidextrous")&(col("mainhand_winner") == "Ambidextrous"), "AA")\
                                   .when((col("mainhand_loser") == "Ambidextrous")&(col("mainhand_winner") == "Right-Handed"), "RA")\
                                   .when((col("mainhand_loser") == "Right-Handed")&(col("mainhand_winner") == "Ambidextrous"), "RA")\
                                   .when((col("mainhand_loser") == "Left-Handed")&(col("mainhand_winner") == "Ambidextrous"), "LA")\
                                   .when((col("mainhand_loser") == "Ambidextrous")&(col("mainhand_winner") == "Left-Handed"), "LA")\
                                   .otherwise("Null"))
                                   


In [68]:
work_df_v2.select(work_df_v2.comb_hands).distinct().show()

23/06/01 20:40:36 WARN TaskSetManager: Stage 280 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.
+----------+
|comb_hands|
+----------+
|        RA|
|        RR|
|        LL|
|        RL|
|      Null|
|        LA|
+----------+



Usar work_df_v2 a partir daqui para dados de jogadores

In [69]:
work_df_v2.where(work_df_v2.GameRank_winner ==work_df_v2.GameRank_loser).count()

23/06/01 20:40:36 WARN TaskSetManager: Stage 283 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


4949

In [70]:
work_df_pandas_v2= work_df_v2.toPandas()

23/06/01 20:40:37 WARN TaskSetManager: Stage 286 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


report = ProfileReport(work_df_pandas_v2)
report.to_file("report_v2.html")

In [71]:
work_df_v2 = work_df_v2.where(work_df_v2.nSets != 1)
work_df_v2 = work_df_v2.where(work_df_v2.nSets != 0)
work_df_v2 = work_df_v2.where(work_df_v2.nSets != 6)



In [72]:
work_df_v2 = work_df_v2.drop(work_df_v2.Sets)

In [73]:
work_df_v2 = work_df_v2.withColumn("rank_dif",abs(work_df_v2.GameRank_winner-work_df_v2.GameRank_loser)) 
  

In [74]:
work_df_v2.show()


23/06/01 20:40:38 WARN TaskSetManager: Stage 287 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.
+--------------------+-----+-------------+------+-----------+---------------+--------------+---------+-------------+---------------+-------------------+--------------+--------+------------+--------------+----------------+-------------+----+-------+----------+-----------+----------+----------+--------+
|           GameRound|nSets|id_tournament|Ground|      Prize|GameRank_winner|GameRank_loser|id_winner|Height_winner|mainhand_winner|    backhand_winner|country_winner|id_loser|Height_loser|mainhand_loser|  backhand_loser|country_loser|Year|is_slam|Height_dif|winner_home|loser_home|comb_hands|rank_dif|
+--------------------+-----+-------------+------+-----------+---------------+--------------+---------+-------------+---------------+-------------------+--------------+--------+------------+--------------+----------------+-------------+----+-------+---

Report v3

work_df_v3 = work_df_v2.toPandas()

report = ProfileReport(work_df_v3)
report.to_file("report_v3.html")

Ver que 

### melissa e tomas

In [75]:
work_dfmt = work_dfmt.withColumn('GameRank_winner', regexp_replace(work_dfmt.GameRank, '[-]', "2300").cast('integer'))
work_dfmt = work_dfmt.withColumn('GameRank_loser', regexp_replace(work_dfmt.GameRank, '[-]', "2300").cast('integer'))

In [76]:
df_filtered = work_dfmt.filter(col("GameRank_winner").isNotNull())

In [77]:
df_filtered_loser = work_dfmt.filter(col("GameRank_loser").isNotNull())


In [78]:
avg_ranks = df_filtered.groupBy("winner_id").agg(avg("GameRank_winner").alias("avg_rank"))


In [79]:
avg_ranks_loser= df_filtered_loser.groupBy("loser_id").agg(avg("GameRank_loser").alias("avg_rank"))

In [80]:
rank_winner = avg_ranks.select(avg_ranks.avg_rank).collect()
winner = avg_ranks.select(avg_ranks.winner_id).collect()

23/06/01 20:40:42 WARN TaskSetManager: Stage 293 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.
23/06/01 20:40:48 WARN TaskSetManager: Stage 326 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.


In [81]:
rank_loser = avg_ranks_loser.select(avg_ranks_loser.avg_rank).collect()
loser = avg_ranks_loser.select(avg_ranks_loser.loser_id).collect()

23/06/01 20:40:52 WARN TaskSetManager: Stage 334 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.
23/06/01 20:40:58 WARN TaskSetManager: Stage 367 contains a task of very large size (4372 KiB). The maximum recommended task size is 1000 KiB.


In [82]:
ranks = []
players_ids = []
for i in range (0,len(rank_loser)):
    rank = rank_loser[i]["avg_rank"]
    id_player = loser[i]["loser_id"]
    ranks.append(rank)
    players_ids.append(id_player)

for i in range (0,len(rank_winner)):
    rank = rank_winner[i]["avg_rank"]
    id_player = winner[i]["winner_id"]
    ranks.append(rank)
    players_ids.append(id_player)


In [83]:
dados = {"player_id" :players_ids, "avg_rank": ranks}
df_pd = pd.DataFrame(dados)


In [84]:
avg_ranks_v2 = spark.createDataFrame(df_pd)


In [85]:
avg_ranks = avg_ranks_v2.groupBy("player_id").agg(avg("avg_rank").alias("avg_rank"))

In [86]:
df = work_df_v2.join(avg_ranks, work_df_v2["id_winner"] == avg_ranks["player_id"], "left") \
       .withColumn("GameRank_winner", when(col("GameRank_winner") == "NaN", col("avg_rank")).otherwise(col("GameRank_winner"))) \
       .drop("avg_rank", "player_id")


In [87]:
df = df.join(avg_ranks, df["id_loser"] == avg_ranks["player_id"], "left") \
       .withColumn("GameRank_loser", when(col("GameRank_loser") == "NaN", col("avg_rank")).otherwise(col("GameRank_loser"))) \
       .drop("avg_rank")


In [88]:
avg_ranks.where(avg_ranks.player_id ==10024).show()


+---------+-----------------+
|player_id|         avg_rank|
+---------+-----------------+
|    10024|937.8693181818181|
+---------+-----------------+



In [89]:
work_df_v2 = df

In [90]:
work_df_v2.count()

23/06/01 20:41:00 WARN TaskSetManager: Stage 373 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


85064

In [91]:
work_df_v2 = work_df_v2.withColumn("rank_dif",abs(work_df_v2.GameRank_winner-work_df_v2.GameRank_loser) ) 

In [92]:
work_df_v2 = work_df_v2.drop(work_df_v2.id_winner)
work_df_v2 = work_df_v2.drop(work_df_v2.id_loser)

In [93]:
work_df_v2 = work_df_v2.drop("player_id")

In [94]:
work_df_v2.filter(work_df_v2.rank_dif.isNull()).count()

23/06/01 20:41:01 WARN TaskSetManager: Stage 382 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


1

In [95]:
work_df_v2.filter(work_df_v2.Prize.isNull()).count()

23/06/01 20:41:03 WARN TaskSetManager: Stage 391 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


390

In [96]:
work_df_v2 = work_df_v2.filter(work_df_v2.rank_dif.isNotNull())

In [97]:
work_df_v2 = work_df_v2.filter(work_df_v2.Prize.isNotNull())

In [98]:
work_df_v2.count()

23/06/01 20:41:04 WARN TaskSetManager: Stage 400 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


84673

In [99]:
work_df_v2.filter(work_df_v2.Height_loser >= 210).count()

23/06/01 20:41:05 WARN TaskSetManager: Stage 409 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


227

In [100]:
work_df_v2.filter(work_df_v2.Height_loser >= 215).count()

23/06/01 20:41:06 WARN TaskSetManager: Stage 418 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


7

In [101]:
work_df_v2.filter(work_df_v2.Height_winner >= 210).count()

23/06/01 20:41:06 WARN TaskSetManager: Stage 427 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


306

In [102]:
work_df_v2.filter(work_df_v2.Height_winner >= 215).count()

23/06/01 20:41:07 WARN TaskSetManager: Stage 436 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


2

In [103]:
work_df_v2.filter(work_df_v2.Height_loser <= 140).count()

23/06/01 20:41:08 WARN TaskSetManager: Stage 445 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


712

In [104]:
work_df_v2.filter(work_df_v2.Height_winner <= 140).count()

23/06/01 20:41:09 WARN TaskSetManager: Stage 454 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


501

In [105]:
work_df_v2 = work_df_v2.filter((work_df_v2.Height_winner <= 215)|(work_df_v2.Height_winner.isNull()))

In [106]:
work_df_v2 = work_df_v2.filter((work_df_v2.Height_loser <= 215)|(work_df_v2.Height_loser.isNull()))

In [107]:
work_df_v2 = work_df_v2.filter((work_df_v2.Height_loser >= 150)|(work_df_v2.Height_loser.isNull()))

In [108]:
work_df_v2 = work_df_v2.filter((work_df_v2.Height_winner >= 150)|(work_df_v2.Height_winner.isNull()))

In [109]:
work_df_v2.count()

23/06/01 20:41:10 WARN TaskSetManager: Stage 463 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.


83459

In [110]:

work_df_v2 = work_df_v2.drop(work_df_v2.Height_winner)
work_df_v2 = work_df_v2.drop(work_df_v2.mainhand_winner)
work_df_v2 = work_df_v2.drop(work_df_v2.backhand_winner)
work_df_v2 = work_df_v2.drop(work_df_v2.country_winner)

work_df_v2 = work_df_v2.drop(work_df_v2.Height_loser)
work_df_v2 = work_df_v2.drop(work_df_v2.mainhand_loser)
work_df_v2 = work_df_v2.drop(work_df_v2.backhand_loser)
work_df_v2 = work_df_v2.drop(work_df_v2.country_loser)




work_df_v3 = work_df_v2.toPandas()

report = ProfileReport(work_df_v3)
report.to_file("report_v3.html")

In [111]:
model_df = work_df_v2

In [112]:
model_df = model_df.where(model_df.is_slam == 0)


In [113]:
model_df = model_df.where(model_df.nSets <= 3)

In [114]:
model_df_auto = model_df

auto_model

In [115]:
model_df_auto.show()

23/06/01 20:41:11 WARN TaskSetManager: Stage 472 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.
+--------------------+-----+-------------+------+---------+---------------+------------------+----+-------+----------+-----------+----------+----------+------------------+
|           GameRound|nSets|id_tournament|Ground|    Prize|GameRank_winner|    GameRank_loser|Year|is_slam|Height_dif|winner_home|loser_home|comb_hands|          rank_dif|
+--------------------+-----+-------------+------+---------+---------------+------------------+----+-------+----------+-----------+----------+----------+------------------+
|         Round of 16|    2|         2932|  Hard|6134605.0|           14.0| 937.8693181818181|2016|      0|      null|          0|         0|      Null| 923.8693181818181|
|         Round of 32|    2|        10471|  Clay|  10000.0|          625.0|             621.0|2012|      0|      null|          0|         0|      Null|               4.

In [116]:
model_df_auto = model_df_auto.drop("is_slam")

In [117]:
model_df_auto = model_df_auto.drop("id_tournament")

In [119]:
model_df_auto.show(1,vertical = True)

23/06/01 20:41:13 WARN TaskSetManager: Stage 484 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.
-RECORD 0----------------------------
 GameRound       | Round of 16       
 nSets           | 2                 
 Ground          | Hard              
 Prize           | 6134605.0         
 GameRank_winner | 14.0              
 GameRank_loser  | 937.8693181818181 
 Year            | 2016              
 Height_dif      | null              
 winner_home     | 0                 
 loser_home      | 0                 
 comb_hands      | Null              
 rank_dif        | 923.8693181818181 
only showing top 1 row



work_df_v3 = model_df_auto.toPandas()

report = ProfileReport(work_df_v3)
report.to_file("report_final.html")

In [424]:
model_df_auto.repartition(1).write.option("header",True).csv("./model_data")

23/05/30 12:27:15 WARN TaskSetManager: Stage 1394 contains a task of very large size (1460 KiB). The maximum recommended task size is 1000 KiB.
